In [1]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [2]:
# %cd /content/drive/MyDrive/Power\ Grid\ Optimization

In [3]:
# !pip install torch
# !pip install torch-geometric
# !pip install pandas openpyxl
# !pip install pandapower

In [4]:
# !pip install -e .

In [5]:
# %cd /content/drive/MyDrive/Power\ Grid\ Optimization/powergrid_torch/data/14Bus/

In [6]:
import numpy as np
import pandas as pd
import torch
from powergrid_torch.utils import pre_processing
import os
from powergrid_torch.models.gnn import GNNModel
from powergrid_torch.models.transformer import GraphTransformerModel
from powergrid_torch.training import train
from powergrid_torch.utils.extras import summary

In [7]:
path_data = os.path.join(os.getcwd(), r'../data/14Bus/')
train_df = pd.read_excel(os.path.join(path_data, 'PF_Dataset_1.xlsx')).values
val_df = pd.read_excel(os.path.join(path_data, 'PF_Dataset_2.xlsx')).values

In [8]:
stats_result = pre_processing.stats(train_df, val_df, 1., 1., 14)

for idx, val in enumerate(stats_result):
    val = val.to(device='cuda')
    stats_result[idx] = val
    
y_norm_train = stats_result[1]
y_train_mean = stats_result[3]
y_train_std = stats_result[5]

y_norm_val = stats_result[7]
y_val_mean = stats_result[9]
y_val_std = stats_result[11]


train_loader, val_loader = pre_processing.main(train_df, val_df, 1., 1., 14)

--------------------Data preparation completed successfully--------------------


d:\softwares\anaconda\anaconda3\envs\raikou\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [9]:
device = pre_processing.get_default_device()
print(device)

cuda


In [10]:
for x in train_loader:
    print(x)
    break

DataBatch(x=[224, 5], edge_index=[2, 480], y=[224, 2], batch=[224], ptr=[17])


In [ ]:
train_loader = pre_processing.DeviceDataLoader(train_loader, device)
val_loader = pre_processing.DeviceDataLoader(val_loader, device)

def model_run(gnn_type = 'GCN', n_epochs = 5, lambda_l2 = 1e-6, n_bus = 14):
    
    if (gnn_type != 'GraphTransformer'):
        model = pre_processing.to_device(GNNModel(in_channels=5, hidden_channels=8, out_channels=2, n_bus=14, gnn_type=gnn_type, is_batch_norm='False', is_dropout=0), device)

    elif (gnn_type == 'GraphTransformer'):
        model = pre_processing.to_device(GraphTransformerModel(in_channels=5, hidden_channels=8, out_channels=2, n_transformer_layers=2, n_heads=3, concat_heads=True), device)


    optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, weight_decay=lambda_l2)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20, verbose=True)
    
    summary(model)
    history = train.fit(n_epochs, 0.001, model, train_loader, val_loader, optimizer, scheduler, y_train_mean, y_train_std, y_val_mean, y_val_std, n_bus)
    return history


In [12]:
history_transformer = model_run("GraphTransformer", n_epochs=10)    

d:\softwares\anaconda\anaconda3\envs\raikou\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


--- Model Summary ---
GraphTransformerModel(
  (transformer_layers): ModuleList(
    (0): TransformerConv(5, 8, heads=3)
    (1): TransformerConv(24, 8, heads=3)
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.0, inplace=False)
  (final_projection): Linear(in_features=24, out_features=2, bias=True)
)


--- Detailed Parameter Summary ---
transformer_layers.0.lin_key.weight                [24, 5]              120
transformer_layers.0.lin_key.bias                  [24]                 24
transformer_layers.0.lin_query.weight              [24, 5]              120
transformer_layers.0.lin_query.bias                [24]                 24
transformer_layers.0.lin_value.weight              [24, 5]              120
transformer_layers.0.lin_value.bias                [24]                 24
transformer_layers.0.lin_skip.weight               [24, 5]              120
transformer_layers.0.lin_skip.bias                 [24]                 24
transformer_layers.1.lin_key.weight                [24, 24]

In [13]:
history_gcn = model_run(gnn_type='GCN', n_epochs=10)

--- Model Summary ---
GNNModel(
  (conv1): GCNConv(5, 8)
  (conv2): GCNConv(8, 8)
  (conv_out): GCNConv(8, 2)
  (bn1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (relu2): ReLU()
  (linear): Linear(in_features=112, out_features=28, bias=True)
)


--- Detailed Parameter Summary ---
conv1.bias                                         [8]                  8
conv1.lin.weight                                   [8, 5]               40
conv2.bias                                         [8]                  8
conv2.lin.weight                                   [8, 8]               64
conv_out.bias                                      [2]                  2
conv_out.lin.weight                                [2, 8]               16
bn1.weight                                         [8]                  8
bn1.bias                                           [8]

In [14]:
# history_GraphConv = model_run(gnn_type='GraphConv')

In [15]:
# history_SAGEConv = model_run(gnn_type='SAGEConv')

In [ ]:
# history_GATConv = model_run(gnn_type='GATConv')

In [17]:
# history_ChebConv = model_run(gnn_type='ChebConv')